In [1]:
pip install lazrs

/Users/Ev1/.zshenv:.:1: no such file or directory: /Users/Ev1/.cargo/env

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Users/Ev1/.pyenv/versions/3.11.4/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install open3d

/Users/Ev1/.zshenv:.:1: no such file or directory: /Users/Ev1/.cargo/env
Note: you may need to restart the kernel to use updated packages.


In [2]:
import laspy
import numpy as np
from sklearn import linear_model
import open3d as o3d
import os
import matplotlib.pyplot as plt

In [5]:

def read_las_file(las_path):
    # Read the LAS file
    las_data = laspy.read(las_path)

    # Extract the points and classifications from the LAS file
    points = np.vstack((las_data.x, las_data.y, las_data.z)).transpose()

    # Create an Open3D PointCloud object
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    #return as point cloud
    return pcd


pcd = read_las_file("off-ground-points.las")

In [20]:
o3d.visualization.draw_geometries([pcd])
num = len(pcd.points)

In [39]:
#downsample cloud
downpcd = pcd.voxel_down_sample(voxel_size=0.1)
num2 = len(downpcd.points)
print(f'Number of points after downsampling reduced from {num} to {num2}')
print(f' {100-(num2/num * 100)} % less points')
#o3d.visualization.draw_geometries([downpcd])

Number of points after downsampling reduced from 18919717 to 21
 99.99988900468226 % less points


In [25]:
#statistical outlier removal

cld, ind = downpcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.2)
num3 = len(cld.points)
print(f'reduced from {num2} to {num3} points')
print(f' {100-(num3/num2 * 100)} % less points')

reduced from 3305786 to 2587112 points
 21.739882738931072 % less points


In [40]:
import open3d as o3d
import numpy as np

# Assuming `pcd` is your point cloud loaded into an o3d.geometry.PointCloud object
points = np.asarray(cld.points)

# define slice levels
z_levels = [0.1, 0.4, 0.7]  

# Create an empty point cloud for the slices
slices_pcd = o3d.geometry.PointCloud()

for z in z_levels:
    # Define a threshold for the slice thickness
    z_threshold = 0.05
    # Select points that fall within the Z level +/- threshold
    slice_mask = (points[:, 2] > (z - z_threshold)) & (points[:, 2] < (z + z_threshold))
    slice_points = points[slice_mask, :3]  # Keep XYZ coordinates for the slice
    
    # Append slice points to the slices_pcd point cloud
    if len(slice_points) > 0:
        slice_pcd = o3d.geometry.PointCloud()
        slice_pcd.points = o3d.utility.Vector3dVector(slice_points)
        slices_pcd += slice_pcd  # Merge the slice into the main slices point cloud

# Visualize all slices together
o3d.visualization.draw_geometries([slices_pcd])


In [10]:
for i, slice_points in enumerate(slices):
    # Create a point cloud object for the slice
    slice_pcd = o3d.geometry.PointCloud()
    slice_pcd.points = o3d.utility.Vector3dVector(np.hstack((slice_points, np.zeros((slice_points.shape[0], 1))))) # Add back Z coordinate as 0
    # Visualize the slice
    print(f"Visualizing slice at Z level: {z_levels[i]}")
    o3d.visualization.draw_geometries([slice_pcd])


Visualizing slice at Z level: 0.5
Visualizing slice at Z level: 1.5
Visualizing slice at Z level: 2.5


In [41]:
# We find clusters
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(slices_pcd.cluster_dbscan(eps=0.5, min_points=5, print_progress=True))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
slices_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 368                     ] 2%
Precompute neighbors.[================>           point cloud has 368 clusters
Precompute neighbors.[========================================] 100%


In [42]:
o3d.visualization.draw_geometries([slices_pcd])

In [36]:
# Get individual point clouds
point_clouds = []
for cluster_label in range(max_label + 1):
    
    cluster_indices = np.where(labels == cluster_label)[0]
    
    cluster_points = slices_pcd.select_by_index(cluster_indices)

    #We only include the individual point cloud if over a certain size
    cluster_size = len(cluster_points.points)
    if(20< cluster_size < 100):
        point_clouds.append(cluster_points)
len(point_clouds)

161

In [43]:
# Initialize an empty point cloud for the combined data
combined_cloud = o3d.geometry.PointCloud()

# Iterate over each individual point cloud and append its data to the combined point cloud
for pcd in point_clouds:
    combined_cloud += pcd

# Visualize the combined point cloud
o3d.visualization.draw_geometries([combined_cloud])



In [44]:


# Initialize an empty list to store 2D points of all clusters
combined_2d_points = []

# Iterate over each individual point cloud
for pcd in point_clouds:
    # Convert to numpy array
    points = np.asarray(pcd.points)
    # Flatten the Z-coordinate
    points_2d = points[:, :2]
    combined_2d_points.append(points_2d)

# Combine all 2D points
combined_2d_points = np.vstack(combined_2d_points)

# Create a new point cloud for the 2D points
combined_2d_cloud = o3d.geometry.PointCloud()
combined_2d_cloud.points = o3d.utility.Vector3dVector(np.hstack((combined_2d_points, np.zeros((combined_2d_points.shape[0], 1)))))

# Visualize the 2D accumulated point cloud
o3d.visualization.draw_geometries([combined_2d_cloud])


In [45]:
import numpy as np
from sklearn import linear_model
from scipy.optimize import least_squares

# Define the function to estimate the circle parameters from three points
def estimate_circle(params, x, y):
    x_c, y_c, r = params
    return np.sqrt((x - x_c)**2 + (y - y_c)**2) - r

# Define the circle model for RANSAC
class CircleModel:
    def __init__(self):
        self.params = None

    def estimate(self, X):
        # Estimate the circle parameters from three points
        x1, y1 = X[0]
        x2, y2 = X[1]
        x3, y3 = X[2]

        # Solving the circle equation using least squares optimization
        # Initial guess for circle center is the mean of the points, with a radius equal to the mean distance to the center
        x_c = np.mean([x1, x2, x3])
        y_c = np.mean([y1, y2, y3])
        r_initial = np.mean(np.sqrt((x_c - [x1, x2, x3])**2 + (y_c - [y1, y2, y3])**2))

        result = least_squares(estimate_circle, x0=np.array([x_c, y_c, r_initial]), args=(np.array([x1, x2, x3]), np.array([y1, y2, y3])))
        
        self.params = result.x
        return True

    def residuals(self, X):
        # Compute the distance to the circle for each point
        x_c, y_c, r = self.params
        x = X[:, 0]
        y = X[:, 1]
        return np.abs(np.sqrt((x - x_c)**2 + (y - y_c)**2) - r)

    def predict_xy(self, r):
        # Return the circle points (for visualization)
        circle = plt.Circle((self.params[0], self.params[1]), self.params[2], color='blue', fill=False)
        return circle

# Use RANSAC to fit the circle model
ransac = linear_model.RANSACRegressor(base_estimator=CircleModel(), min_samples=3, residual_threshold=0.1, max_trials=1000)

# Assuming `combined_2d_points` is a Nx2 array with your 2D points
ransac.fit(combined_2d_points, np.zeros(combined_2d_points.shape[0]))

# Extract the circle parameters
circle_params = ransac.estimator_.params
print(f"Estimated circle parameters: Center ({circle_params[0]}, {circle_params[1]}), Radius {circle_params[2]}")

# For visualization, you can use matplotlib to draw the circle
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
# Plot the 2D points
ax.scatter(combined_2d_points[:, 0], combined_2d_points[:, 1], color='black')
# Plot the detected circle
circle = plt.Circle((circle_params[0], circle_params[1]), circle_params[2], color='blue', fill=False)
ax.add_artist(circle)
ax.set_aspect('equal', 'box')
plt.show()


TypeError: RANSACRegressor.__init__() got an unexpected keyword argument 'base_estimator'